In [2]:
import pandas as pd

In [3]:
pd.__version__

'2.2.2'

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [8]:
df_jan=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [ ]:
df_feb=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
df_jan

In [ ]:
df_jan.shape

In [ ]:
df_jan.duration.describe()

In [ ]:
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [ ]:
df_jan.shape

In [ ]:
df_feb.shape

In [ ]:
fract_after_drop_outliers=(3009173/3066766)*100
fract_after_drop_outliers

In [ ]:
df_jan['PU_DO'] = df_jan['PULocationID'] + '_' + df_jan['DOLocationID']
df_feb['PU_DO'] = df_feb['PULocationID'] + '_' + df_feb['DOLocationID']

In [ ]:
categorical = ['PU_DO']
numerical = ['trip_distance']


In [ ]:
train_dicts = df_jan[categorical + numerical].to_dict(orient='records')
val_dicts = df_feb[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_jan[target].values
y_val = df_feb[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)